<a href="https://colab.research.google.com/github/GauravAero/Investment_Advisor/blob/main/Investment_Advisor_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comprehensive Investment Advisory Workflow Using CrewAI


In this notebook, we illustrate a sophisticated investment advisory system built using **CrewAI**. The workflow combines multiple AI agents, each equipped with specialized tools, to:
1. Gather financial data and news about a company.
2. Analyze the data comprehensively.
3. Provide actionable investment recommendations.

---
### Workflow Overview
1. Define custom tools to retrieve financial data, income statements, and stock prices.
2. Create agents for data gathering, analysis, and recommendation tasks.
3. Define tasks aligned with these agents' roles and objectives.
4. Configure a crew to execute these tasks sequentially, leveraging collaboration between agents.
5. Run the workflow and observe the results.

---
### Code Walkthrough
Below is the implementation.


### Step 1: Import Required Libraries and Declare in-built Tools

In [1]:
## initial setup for Colab.
## if you are using this in local jupyter notebook, skip this cell
!pip install crewai
!pip install crewai-tools
!pip install duckduckgo-search
!pip install yfinance
!pip install curl_cffi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.7/366.7 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.2 MB/s eta 0

In [2]:
from crewai import Agent, Task
from crewai.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from datetime import datetime

# Current date for context
Now = datetime.now()
Today = Now.strftime("%d-%b-%Y")

# Define a web search tool
@tool("DuckDuckGo Search")
def search_tool(search_query: str):
    """Search the internet for information on a given topic"""
    return DuckDuckGoSearchRun().run(search_query)

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Step 2: Define Custom Tools

In [7]:
from crewai.tools import tool
import json

## Import Yahoo Finance APIs
import yfinance as yf
from curl_cffi import requests
import time

session = requests.Session(impersonate="chrome")

## A function is defined, that will work as a tool and that is provided to the framework (hence to agents) as a tool with the '@tool' decorator
## Note the function description  (purpose, usage) in doc strigs.
@tool ("Get current stock price")
def get_current_stock_price(symbol: str) -> str:
    """Use this function to get the current stock price for a given symbol.

    Args:
        symbol (str): The stock symbol.

    Returns:
        str: The current stock price or error message.
    """
    try:
        time.sleep(0.5)
        stock = yf.Ticker(symbol,session=session)

        current_price = stock.info.get("regularMarketPrice", stock.info.get("currentPrice"))
        return f"{current_price:.2f}" if current_price else f"Could not fetch current price for {symbol}"
    except Exception as e:
        return f"Error fetching current price for {symbol}: {e}"

@tool
def get_company_info(symbol: str):
    """Use this function to get company information and current financial snapshot for a given stock symbol.

    Args:
        symbol (str): The stock symbol.

    Returns:
        JSON containing company profile and current financial snapshot.
    """
    try:
        company_info_full = yf.Ticker(symbol,session=session).info
        if company_info_full is None:
            return f"Could not fetch company info for {symbol}"

        company_info_cleaned = {
            "Name": company_info_full.get("shortName"),
            "Symbol": company_info_full.get("symbol"),
            "Current Stock Price": f"{company_info_full.get('regularMarketPrice', company_info_full.get('currentPrice'))} {company_info_full.get('currency', 'USD')}",
            "Market Cap": f"{company_info_full.get('marketCap', company_info_full.get('enterpriseValue'))} {company_info_full.get('currency', 'USD')}",
            "Sector": company_info_full.get("sector"),
            "Industry": company_info_full.get("industry"),
            "City": company_info_full.get("city"),
            "Country": company_info_full.get("country"),
            "EPS": company_info_full.get("trailingEps"),
            "P/E Ratio": company_info_full.get("trailingPE"),
            "52 Week Low": company_info_full.get("fiftyTwoWeekLow"),
            "52 Week High": company_info_full.get("fiftyTwoWeekHigh"),
            "50 Day Average": company_info_full.get("fiftyDayAverage"),
            "200 Day Average": company_info_full.get("twoHundredDayAverage"),
            "Employees": company_info_full.get("fullTimeEmployees"),
            "Total Cash": company_info_full.get("totalCash"),
            "Free Cash flow": company_info_full.get("freeCashflow"),
            "Operating Cash flow": company_info_full.get("operatingCashflow"),
            "EBITDA": company_info_full.get("ebitda"),
            "Revenue Growth": company_info_full.get("revenueGrowth"),
            "Gross Margins": company_info_full.get("grossMargins"),
            "Ebitda Margins": company_info_full.get("ebitdaMargins"),
        }
        return json.dumps(company_info_cleaned)
    except Exception as e:
        return f"Error fetching company profile for {symbol}: {e}"

@tool
def get_income_statements(symbol: str):

    """Use this function to get income statements for a given stock symbol.

    Args:
    symbol (str): The stock symbol.

    Returns:
    JSON containing income statements or an empty dictionary.
    """
    try:
        stock = yf.Ticker(symbol,session=session)
        financials = stock.financials
        return financials.to_json(orient="index")
    except Exception as e:
        return f"Error fetching income statements for {symbol}: {e}"


### Step 3: Define the Agents



In [8]:
from crewai import Agent, Task, Crew, Process
import os
from google.colab import userdata

# Retrieve your API key from Colab secrets
os.environ["GOOGLE_API_KEY"] = userdata.get('Gemini_key')

# Use CrewAI's built-in LLM configuration (Recommended)
from crewai import LLM

# Configure Gemini LLM directly through CrewAI
gemini_llm = LLM(
    model="gemini/gemini-2.5-flash-lite",  # Use gemini/ prefix for CrewAI
    api_key=os.environ["GOOGLE_API_KEY"]
)
# Agent for gathering company news and information
news_info_explorer = Agent(
    role='News and Info Researcher',
    goal='Gather and provide the latest news and information about a company from the internet',
    #llm='gpt-4o',
    llm = gemini_llm,
    verbose=True,
    backstory=(
        'You are an expert researcher, who can gather detailed information about a company. '
        'Consider you are on: ' + Today
    ),
    tools=[search_tool],
    cache=True,
    max_iter=5,
)

# Agent for gathering financial data
data_explorer = Agent(
    role='Data Researcher',
    goal='Gather and provide financial data and company information about a stock',
    #llm='gpt-4o',
    llm = gemini_llm,
    verbose=True,
    backstory=(
        'You are an expert researcher, who can gather detailed information about a company or stock. '
        'When using tools, use the stock symbol and add a suffix ".NS" to it. try with and without the suffix and see what works'
        'Consider you are on: ' + Today
    ),
    tools=[get_company_info, get_income_statements],
    cache=True,
    max_iter=5,
)

# Agent for analyzing data
analyst = Agent(
    role='Data Analyst',
    goal='Consolidate financial data, stock information, and provide a summary',
    #llm='gpt-4o',
    llm = gemini_llm,
    verbose=True,
    backstory=(
        'You are an expert in analyzing financial data, stock/company-related current information, and '
        'making a comprehensive analysis. Use Indian units for numbers (lakh, crore). '
        'Consider you are on: ' + Today
    ),
)

# Agent for financial recommendations
fin_expert = Agent(
    role='Financial Expert',
    goal='Considering financial analysis of a stock, make investment recommendations',
    #llm='gpt-4o',
    llm = gemini_llm,
    verbose=True,
    tools=[get_current_stock_price],
    max_iter=5,
    backstory=(
        'You are an expert financial advisor who can provide investment recommendations. '
        'Consider the financial analysis, current information about the company, current stock price, '
        'and make recommendations about whether to buy/hold/sell a stock along with reasons.'
        'When using tools, try with and without the suffix ".NS" to the stock symbol and see what works. '
        'Consider you are on: ' + Today
    ),
)

### Step 4: Define the Tasks

In [9]:
from crewai import Task

# Task to gather financial data of a stock
get_company_financials = Task(
    description="Get financial data like income statements and other fundamental ratios for stock: {stock}",
    expected_output="Detailed information from income statement, key ratios for {stock}. "
                    "Indicate also about current financial status and trend over the period.",
    agent=data_explorer,
)

# Task to gather company news
get_company_news = Task(
    description="Get latest news and business information about company: {stock}",
    expected_output="Latest news and business information about the company. Provide a summary also.",
    agent=news_info_explorer,
)

# Task to analyze financial data and news
analyse = Task(
    description="Make thorough analysis based on given financial data and latest news of a stock",
    expected_output="Comprehensive analysis of a stock outlining financial health, stock valuation, risks, and news. "
                    "Mention currency information and number units in Indian context (lakh/crore).",
    agent=analyst,
    context=[get_company_financials, get_company_news],
    output_file='Analysis.md',
)

# Task to provide financial advice
advise = Task(
    description="Make a recommendation about investing in a stock, based on analysis provided and current stock price. "
                "Explain the reasons.",
    expected_output="Recommendation (Buy / Hold / Sell) of a stock backed with reasons elaborated."
                    "Response in Mark down format.",
    agent=fin_expert,
    context=[analyse],
    output_file='Recommendation.md',
)

### Step 5: Set Up the Crew

In [10]:
from crewai import Crew, Process
from datetime import datetime

# Callback function to print a timestamp
def timestamp(Input):
    print(datetime.now())

# Define the crew with agents and tasks in sequential process
crew = Crew(
    agents=[data_explorer, news_info_explorer, analyst, fin_expert],
    tasks=[get_company_financials, get_company_news, analyse, advise],
    verbose=True,
    Process=Process.sequential,
    step_callback=timestamp,
)

### Step 5: Run the Crew and Observe Results

In [12]:
# Run the crew with a specific stock
result = crew.kickoff(inputs={'stock': 'OLAELEC'})

# Print the final result
print("Final Result:", result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 78917c2f-3876-4229-94ab-1bc15cd0cec0                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Researcher                                                                                         │
│                                                                                                                 │
│  Task: Get financial data like income statements and other fundamental ratios for stock: OLAELEC                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:112: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  self.msg_id = ip.kernel._parent_header['header']['msg_id']

...

sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x790134eff5b0>


╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Financial Expert                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Sell**                                                                                                       │
│                                                                                                                 │
│  **Reasons:**                                                                                                   │
│                                                                                                                 │
│  Ola Electric Mobility Ltd (OLAELEC.NS) presents a high-risk investment profile based on the comprehensive      │
│  analysis. The company is currently deeply unprofitable, as evidenced by a negative Earnings Per Share (EPS)    │
│  of -₹5.64 INR and consistent, substantial net losses across all reported periods, with the latest period       │
│  showing a net loss of ₹2,276 crore.                                                                            │
│                                                                                                                 │
│  A significant concern is the sharp 49.6% year-over-year decline in revenue from FY23 to FY24. This             │
│  contraction, coupled with escalating operating expenses and increasing interest expenses, contributes to       │
│  deeply negative operating and net profit margins (-51.06% and -50.42% respectively for FY24). While gross      │
│  margins have shown some improvement, they are not sufficient to offset the high overheads and financing        │
│  costs.                                                                                                         │
│                                                                                                                 │
│  The current market capitalization of approximately ₹17,072.6 crore appears disproportionately high given the   │
│  company's current financial performance, suggesting that the valuation is heavily reliant on future growth     │
│  potential that has yet to materialize. The company faces considerable risks related to its ability to achieve  │
│  profitability, manage its cash burn, and execute its expansion plans in an increasingly competitive market.    │
│  Until the company can demonstrate a clear path to profitability and reverse the trend of revenue decline,      │
│  investing in Ola Electric Mobility Ltd carries a significant risk of capital loss.                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 0b6eb14d-1a04-492b-84ac-cf7a115cf545                                                                     │
│  Agent: Financial Expert                                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 78917c2f-3876-4229-94ab-1bc15cd0cec0                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: **Sell**                                                                                         │
│                                                                                                                 │
│  **Reasons:**                                                                                                   │
│                                                                                                                 │
│  Ola Electric Mobility Ltd (OLAELEC.NS) presents a high-risk investment profile based on the comprehensive      │
│  analysis. The company is currently deeply unprofitable, as evidenced by a negative Earnings Per Share (EPS)    │
│  of -₹5.64 INR and consistent, substantial net losses across all reported periods, with the latest period       │
│  showing a net loss of ₹2,276 crore.                                                                            │
│                                                                                                                 │
│  A significant concern is the sharp 49.6% year-over-year decline in revenue from FY23 to FY24. This             │
│  contraction, coupled with escalating operating expenses and increasing interest expenses, contributes to       │
│  deeply negative operating and net profit margins (-51.06% and -50.42% respectively for FY24). While gross      │
│  margins have shown some improvement, they are not sufficient to offset the high overheads and financing        │
│  costs.                                                                                                         │
│                                                                                                                 │
│  The current market capitalization of approximately ₹17,072.6 crore appears disproportionately high given the   │
│  company's current financial performance, suggesting that the valuation is heavily reliant on future growth     │
│  potential that has yet to materialize. The company faces considerable risks related to its ability to achieve  │
│  profitability, manage its cash burn, and execute its expansion plans in an increasingly competitive market.    │
│  Until the company can demonstrate a clear path to profitability and reverse the trend of revenue decline,      │
│  investing in Ola Electric Mobility Ltd carries a significant risk of capital loss.                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final Result: **Sell**

**Reasons:**

Ola Electric Mobility Ltd (OLAELEC.NS) presents a high-risk investment profile based on the comprehensive analysis. The company is currently deeply unprofitable, as evidenced by a negative Earnings Per Share (EPS) of -₹5.64 INR and consistent, substantial net losses across all reported periods, with the latest period showing a net loss of ₹2,276 crore.

A significant concern is the sharp 49.6% year-over-year decline in revenue from FY23 to FY24. This contraction, coupled with escalating operating expenses and increasing interest expenses, contributes to deeply negative operating and net profit margins (-51.06% and -50.42% respectively for FY24). While gross margins have shown some improvement, they are not sufficient to offset the high overheads and financing costs.

The current market capitalization of approximately ₹17,072.6 crore appears disproportionately high given the company's current financial performance, suggesting that the valuation is h


---
### Conclusion
This workflow demonstrates a detailed investment advisory process using multiple agents and tasks. The system showcases the integration of custom tools and collaborative agents in CrewAI to provide actionable financial insights.
